## DIA 097: Enviar el Reporte PDF por Correo al Usuario (Flask + smtplib)


Hoy aprenderás a enviar el reporte PDF generado por correo electrónico al usuario logueado.
Esto permite automatizar notificaciones, compartir reportes sin descargas manuales y dar un toque profesional a tu app.

✅ ¿Qué aprenderás hoy?

📌 Configurar envío de correos en Flask.

📌 Adjuntar archivos PDF a un correo.

📌 Usar las credenciales de un correo SMTP seguro (ej. Gmail).

🧩 1. Requisitos Previos
Instala si no lo tienes:

bash
Copiar
Editar
pip install python-dotenv
Crea un archivo .env con tus credenciales:

env
Copiar
Editar
EMAIL_SENDER=tu_correo@gmail.com
EMAIL_PASSWORD=tu_contraseña_app  # Usa contraseña de aplicación de Gmail
⚠️ Importante: Usa contraseñas de aplicación, no la de tu cuenta de Gmail directamente.

🧩 2. Configuración en app.py
Primero importa:

python
Copiar
Editar
import smtplib
import os
from email.message import EmailMessage
from dotenv import load_dotenv

load_dotenv()
🧩 3. Nueva Ruta /enviar-pdf-email
python
Copiar
Editar
@app.route('/enviar-pdf-email', methods=['GET', 'POST'])
@login_required
def enviar_pdf_email():
    if request.method == 'POST':
        fecha_inicio = request.form['fecha_inicio']
        fecha_fin = request.form['fecha_fin']

        try:
            f1 = datetime.strptime(fecha_inicio, '%Y-%m-%d')
            f2 = datetime.strptime(fecha_fin, '%Y-%m-%d')

            registros = Registro.query.filter(
                Registro.user_id == current_user.id,
                Registro.fecha >= f1,
                Registro.fecha <= f2
            ).order_by(Registro.fecha.desc()).all()

            if not registros:
                return "⚠️ No se encontraron registros en ese rango.", 404

            # Generar PDF en memoria
            buffer = io.BytesIO()
            p = canvas.Canvas(buffer, pagesize=letter)
            width, height = letter

            p.setFont("Helvetica-Bold", 14)
            p.drawString(50, height - 50, f"Reporte de {current_user.username}")
            p.setFont("Helvetica", 12)
            p.drawString(50, height - 70, f"Desde {fecha_inicio} hasta {fecha_fin}")

            y = height - 100
            p.setFont("Helvetica", 10)
            p.drawString(50, y, "Fecha        | F1 | F2 | F3 | F4 | Predicción")
            y -= 15
            p.line(50, y, width - 50, y)
            y -= 15

            for r in registros:
                linea = f"{r.fecha.strftime('%Y-%m-%d %H:%M')}  | {r.f1}  | {r.f2}  | {r.f3}  | {r.f4}  | {r.prediccion}"
                p.drawString(50, y, linea)
                y -= 15
                if y < 50:
                    p.showPage()
                    y = height - 50

            p.save()
            buffer.seek(0)

            # Enviar correo
            msg = EmailMessage()
            msg['Subject'] = f'Reporte de predicciones del {fecha_inicio} al {fecha_fin}'
            msg['From'] = os.getenv('EMAIL_SENDER')
            msg['To'] = current_user.email
            msg.set_content(f'Hola {current_user.username},\n\nAdjunto encontrarás tu reporte en PDF entre {fecha_inicio} y {fecha_fin}.\n\nSaludos,\nTu App de Predicción')

            msg.add_attachment(buffer.read(), maintype='application', subtype='pdf',
                               filename=f'reporte_{fecha_inicio}_a_{fecha_fin}.pdf')

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                smtp.login(os.getenv('EMAIL_SENDER'), os.getenv('EMAIL_PASSWORD'))
                smtp.send_message(msg)

            return "📧 Reporte enviado exitosamente al correo registrado."

        except Exception as e:
            return f"❌ Error al enviar el correo: {str(e)}", 500

    return render_template('enviar_pdf_email.html')
🖼️ 4. Template: enviar_pdf_email.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Enviar PDF por Email</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center mb-4">📩 Enviar Reporte PDF a tu Correo</h2>
    <form method="POST" class="row g-3">
        <div class="col-md-5">
            <label class="form-label">Desde:</label>
            <input type="date" class="form-control" name="fecha_inicio" required>
        </div>
        <div class="col-md-5">
            <label class="form-label">Hasta:</label>
            <input type="date" class="form-control" name="fecha_fin" required>
        </div>
        <div class="col-md-2 align-self-end">
            <button type="submit" class="btn btn-primary w-100">Enviar</button>
        </div>
    </form>
    <a href="/mi-historial" class="btn btn-outline-secondary mt-4">← Volver</a>
</div>
</body>
</html>
